<a href="https://colab.research.google.com/github/hmehryar/DeepLearningWithKerasTutorial/blob/main/deep_learning_41_gans_tabular_synthetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    from google.colab import drive
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: using Google CoLab
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
# HIDE OUTPUT
CMD = "wget https://raw.githubusercontent.com/Diyago/"\
  "Tabular-data-generation/master/requirements.txt"

!{CMD}
!pip install -r requirements.txt
!pip install tabgan

--2024-09-25 21:11:37--  https://raw.githubusercontent.com/Diyago/Tabular-data-generation/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230 [text/plain]
Saving to: ‘requirements.txt.1’

requirements.txt.1  100%[===================>]     230  --.-KB/s    in 0s      

2024-09-25 21:11:37 (5.81 MB/s) - ‘requirements.txt.1’ saved [230/230]

  Using cached numpy-1.22.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.0 kB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl.metadata (8.0 kB)
  Using cached be_great-0.0.7-py3-none-any.whl.metadata (5.1 kB)
  Using cached datasets-3.0.0-py3-none-any.whl.metadata (19 kB)
INFO: pip is looking at multiple versions of be-great to determine which version is compatibl

In [ ]:
# HIDE OUTPUT
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA', '?'])

COLS_USED = ['cylinders', 'displacement', 'horsepower', 'weight',
          'acceleration', 'year', 'origin','mpg']
COLS_TRAIN = ['cylinders', 'displacement', 'horsepower', 'weight',
          'acceleration', 'year', 'origin']

df = df[COLS_USED]

# Handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())


# Split into training and test sets
df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(
    df.drop("mpg", axis=1),
    df["mpg"],
    test_size=0.20,
    #shuffle=False,
    random_state=42,
)

# Create dataframe versions for tabular GAN
df_x_test, df_y_test = df_x_test.reset_index(drop=True), \
  df_y_test.reset_index(drop=True)
df_y_train = pd.DataFrame(df_y_train)
df_y_test = pd.DataFrame(df_y_test)

# Pandas to Numpy
x_train = df_x_train.values
x_test = df_x_test.values
y_train = df_y_train.values
y_test = df_y_test.values

# Build the neural network
model = Sequential()
# Hidden 1
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(12, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3,
        patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),
        callbacks=[monitor], verbose=2,epochs=1000)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/1000
10/10 - 2s - 231ms/step - loss: 84494.9453 - val_loss: 20214.9902
Epoch 2/1000
10/10 - 0s - 8ms/step - loss: 6942.5435 - val_loss: 510.1043
Epoch 3/1000
10/10 - 0s - 12ms/step - loss: 267.6839 - val_loss: 574.4251
Epoch 4/1000
10/10 - 0s - 8ms/step - loss: 777.2969 - val_loss: 766.1308
Epoch 5/1000
10/10 - 0s - 7ms/step - loss: 512.6537 - val_loss: 226.0466
Epoch 6/1000
10/10 - 0s - 9ms/step - loss: 135.8732 - val_loss: 87.0802
Epoch 7/1000
10/10 - 0s - 7ms/step - loss: 116.4048 - val_loss: 112.1939
Epoch 8/1000
10/10 - 0s - 8ms/step - loss: 117.4212 - val_loss: 91.2785
Epoch 9/1000
10/10 - 0s - 9ms/step - loss: 100.4432 - val_loss: 85.6667
Epoch 10/1000
10/10 - 0s - 14ms/step - loss: 98.2351 - val_loss: 86.9091
Epoch 11/1000
10/10 - 0s - 10ms/step - loss: 97.3494 - val_loss: 84.7474
Epoch 12/1000
10/10 - 0s - 14ms/step - loss: 96.7594 - val_loss: 84.2739
Epoch 13/1000
10/10 - 0s - 13ms/step - loss: 96.2337 - val_loss: 83.9925
Epoch 14/1000
10/10 - 0s - 15ms/step - loss: 9

In [ ]:
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Final score (RMSE): 3.3160822679083872


In [ ]:

!apt-get install be_great

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package be_great


In [ ]:
from tabgan.sampler import GANGenerator
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

gen_x, gen_y = GANGenerator(gen_x_times=1.1, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999, \
              is_post_process=True,
           adversarial_model_params={
               "metrics": "rmse", "max_depth": 2, "max_bin": 100,
               "learning_rate": 0.02, "random_state": \
                42, "n_estimators": 500,
           }, pregeneration_frac=2, only_generated_data=False,\
           gan_params = {"batch_size": 500, "patience": 25, \
          "epochs" : 500,}).generate_data_pipe(df_x_train, df_y_train,\
          df_x_test, deep_copy=True, only_adversarial=False, \
          use_adversarial=True)

In [ ]:
gen_x


In [ ]:
# Predict
pred = model.predict(gen_x.values)
score = np.sqrt(metrics.mean_squared_error(pred,gen_y.values))
print("Final score (RMSE): {}".format(score))